In [1]:
from multiprocessing import Pool,Process,Manager,cpu_count
import math
import numpy as np
import pylab
import time
from pathlib import Path
import numpy as np
from PyBoltz.PyBoltzRun import *
import pickle
import time 

In [4]:

### Simulation for Pressure range
#Pressure = np.arange(750.062,8250.68,750.062)

In [2]:
def SimulateDiffusion(GasProperties,EField,Pressure,Values):
    # Set up helper object
    PBRun=PyBoltzRun()
    print("Running with E Field " +str(EField) + " V/cm With Pressure " + str(Pressure) + " tor\n")
    GasProperties['EField_Vcm']=EField
    GasProperties['Pressure_Torr']=Pressure
    #print(GasProperties)
    Output=PBRun.Run(GasProperties)
    Key=str(round(EField,2))+"_"+str(round(Pressure,2))
    value={}
    value["Drift_vel"]=[Output['Drift_vel'].val[2],Output['Drift_vel'].err[2]]
    value["DT"]=[Output['DT'].val,Output['DT'].err]
    value["DL"]=[Output['DL'].val,Output['DL'].err]
    value["DT1"]=[Output['DT1'].val,Output['DT1'].err]
    value["DL1"]=[Output['DL1'].val,Output['DL1'].err]
    Values[Key]=value
    del PBRun
    
## run simulation with using multi-thread  
def MultiProcessManager(GasProperties,EFields,Pressures,NProcess,Dict,Delay=5):
    AcceptedValues=[list,np.ndarray]
    if(type(EFields) not in AcceptedValues and type(Pressures) not in AcceptedValues ):
        print("Please Provide list of Efields and Pressures to run MultiProcess Manager")
        return

    ## Create The EField,Presure value Paramater
    Parameters=np.empty((0,2))
  
    ## Simulating Combination Events with Different Pressure and Electrical Fields
    for E in EFields:
        for P in Pressures:
            v=[E,P]
            Parameters=np.vstack((Parameters,v))
            
    MaxNumEvents=len(Parameters)
    
    ## obtain the CPU info
    if(NProcess>int(cpu_count())):
        print(f"Your CPU count is {cpu_count()} less than the Number of Process {NProcess } therefore we are selecting the Number of Process to be {cpu_count()}\n")
        NProcess=int(cpu_count())
        
    print("Total Number of Gas Configuration to Simulate = " +str(len(Parameters)) )
    print("Number of Processes = " +str(NProcess) )

    
    if(MaxNumEvents%NProcess==0):   
        NumOfRuns=(int)(MaxNumEvents/NProcess)
    else:
        NumOfRuns=math.ceil(MaxNumEvents/NProcess)
    print("Number of Runs = " +str(NumOfRuns) )

   
    
    print("Starting the Simulation ....")
    
    EventCount=0
    
    
    Processes=[]
    ## Loop Through of the runs
    NumofEvents=MaxNumEvents
    Dict["Efields"]=EFields
    Dict["Pressures"]=Pressures

    for run in range(0,NumOfRuns):
        print("Starting Run " +str(run+1)+" out of " + str(NumOfRuns))
        
        ### Determine the Number Of Events to Process in each loop
        if(NumofEvents<0): break # break the code if it is negative
            
        reminder=NumofEvents%NProcess
        if (NumofEvents>=NProcess): 
            LoopValue=NProcess 
        else:
            LoopValue=reminder
        print("Expected Number of Events For This Run = " + str(LoopValue))
        
        #Start the process
        for event in range(0,LoopValue):
            Paramater=Parameters[EventCount]  # EField,Pressure
            proc=Process(target=SimulateDiffusion,args=(GasProperties,Paramater[0],Paramater[1],Dict))
            Processes.append(proc)
            time.sleep(Delay) ## Introduce 2 seconds delay so it will help to keep results in order
            proc.start()
            EventCount+=1
        
        # Complete     
        for p in Processes:
            p.join()
            time.sleep(Delay) ## Introduce 2 seconds delay so it will help to keep results in order

        Processes.clear()
            
        NumofEvents=MaxNumEvents-EventCount
        
def SingleProcessManager(GasProperties,EField,Pressure):
    value={}
    # Check to make sure We dnt have list 
    if(type(EField) is list and type(Pressure) is list ):
        print("Array is given!! We are selecting only first index elements for Pressure and Electrical Field!")
        EField=EField[0]
        Pressure=Pressure[0]

    GasProperties['EField_Vcm']=EField
    GasProperties['Pressure_Torr']=Pressure
    print("--- Simulating Single Event with Gas Configuration ---")
    print(GasProperties)
    SimulateDiffusion(GasProperties,EField,Pressure,value)
    return value


def PickleITDump(data,fileName):
    with open(fileName,'wb') as f:
        pickle.dump(data,f)
        
def PickleITRead(fileName):
    
    with open(fileName,'rb') as f:
        data=pickle.load(f)
    return data
def CreateGasMixtures(first,second,percentIncrement,amount,Mode,FileName):
    Events={}
    for i in range(1,amount):
        Event={}
        Event["Gases"]=[first,second]
        Event["Fractions"]=[100-(i*percentIncrement),(i*percentIncrement)]
        if(FileName==None):
            Event["FileName"]="Diffusion_"+str(Event["Gases"][0])+"_"+str(Event["Fractions"][0])+"_"+str(Event["Gases"][1])+"_"+str(Event["Fractions"][1])+"_Mode_"+str(Mode)+".p"
        else:
            Event["FileName"]=FileName+"_"+str(Event["Gases"][0])+"_"+str(Event["Fractions"][0])+"_"+str(Event["Gases"][1])+"_"+str(Event["Fractions"][1])+"_Mode_"+str(Mode)+".p"
        Events[i-1]=Event
    return Events

## Angular Distribution Modes

def GasMixturesMultiProcess(first,second,GasProperties,FilePath,E,P,Nprocess,PercentIncrement,Amount,Mode=0,FileName=None):
    t1=time.time()
    Events=CreateGasMixtures(first,second,PercentIncrement,Amount,Mode,FileName)
    for ev in Events:
        ## Create a Shared Dictionary
        Dict=manager.dict()
        GasProperties["Angular_dist_model"]=Mode
        GasProperties['Gases']=Events[ev]["Gases"]
        GasProperties['Fractions']=Events[ev]["Fractions"]
        MultiProcessManager(GasProperties,E,P,Nprocess,Dict)
        Path=FilePath+Events[ev]["FileName"]
        PickleITDump(dict(Dict),Path)
        del Dict
    t2=time.time()
    print("Time elapsed:" +str(t2-t1))
        
        
def SingleGasMultiProcess(Gas,GasProperties,FilePath,E,P,Nprocess,Delay,Mode=0,FileName=None):
    t1=time.time()
    dd=manager.dict()
    GasProperties["Angular_dist_model"]=Mode
    MultiProcessManager(GasProperties,E,P,Nprocess,dd,Delay=Delay)
    if(FileName==None):
        Path=FilePath+Gas
    else:
        Path=FilePath+FileName
    PickleITDump(dict(dd),Path)
    del dd
    t2=time.time()

    print("Time elapsed:" +str(t2-t1))
    

In [3]:
#Create MultiProcess Manager
manager=Manager()

gpv1   ={'Gases'              :['XENON'],
               'Fractions'             :[100],
               'Max_collisions'        :4e7,
               'EField_Vcm'            :500,
               'Max_electron_energy'   :0,
               'Temperature_C'         :23,
               'Pressure_Torr'         :7500.62,
               'BField_Tesla'          :0,
               'BField_angle'          :0,
               'Angular_dist_model'    :2, # 0 is Isotropic Scattering
               'Enable_penning'        :0,
               'Enable_thermal_motion' :1,
               'ConsoleOutputFlag'     :0,
                'NumSamples'            :30}

#GasMixturesMultiProcess("XENON","CO2",gg,"/home/argon/Projects/Ilker/PyBoltz/Diffusion/",Efieldss,pp,NProcess,0.1,4)


In [4]:
manager=Manager()
#EFieldss=np.linspace(10,500,20)
#EFieldss=np.arange(100,1000,20)
EFieldss=np.arange(100,600,20)
#pp=np.arange(5*760.062,8250.68,760.062)
InitialPressure=750.062 # 1 Bar
pp=[]
for i in range(5,11):
    pp.append(i*InitialPressure)
NProcess=20
print("-- ElectricFields --" )
print(EFieldss)
print("-- Pressures --" )
print(pp)


-- ElectricFields --
[100 120 140 160 180 200 220 240 260 280 300 320 340 360 380 400 420 440
 460 480 500 520 540 560 580]
-- Pressures --
[3750.31, 4500.372, 5250.434, 6000.496, 6750.558, 7500.62]


In [ ]:
#GasMixturesMultiProcess("XENON","SF6",gpv1,"/home/ilker/CRAB/Diff/",EFieldss,pp,NProcess,5,4)
GasMixturesMultiProcess("XENON","CH4",gpv1,"/home/argon/Downloads/Diffusion_Plot/",EFieldss,pp,NProcess,0.1,6,Mode=0,FileName="XenonCH4_Mode2")
#GasMixturesMultiProcess("XENON","CH4",gpv1,"/home/ilker/CRAB/Diff/",EFieldss,pp,NProcess,0.01,4)
#GasMixturesMultiProcess("XENON","HELIUM4",gpv1,"/home/ilker/CRAB/Diff/",EFieldss,pp,NProcess,5,4)
#GasMixturesMultiProcess("XENON","NEON",gpv1,"/home/ilker/CRAB/Diff/",EFieldss,pp,NProcess,5,4)
#GasMixturesMultiProcess("XENON","ETHANE",gpv1,"/home/ilker/CRAB/Diff/",EFieldss,pp,NProcess,5,4)
#GasMixturesMultiProcess("XENON","HYDROGEN",gpv1,"/home/ilker/CRAB/Diff/",EFieldss,pp,NProcess,5,4)
#GasMixturesMultiProcess("XENON","KRYPTON",gpv1,"/home/ilker/CRAB/Diff/",EFieldss,pp,NProcess,5,4)
#GasMixturesMultiProcess("XENON","CO2",gpv1,"/home/ilker/CRAB/Diff/",EFieldss,pp,NProcess,0.01,4)


Total Number of Gas Configuration to Simulate = 150
Number of Processes = 20
Number of Runs = 8
Starting the Simulation ....
Starting Run 1 out of 8
Expected Number of Events For This Run = 20
Running with E Field 100.0 V/cm With Pressure 3750.31 tor

Input Decor_Colls not set, using default 0
Input Decor_LookBacks not set, using default 0
Input Decor_Step not set, using default 0

Running with E Field 100.0 V/cm With Pressure 4500.372 tor

Input Decor_Colls not set, using default 0
Input Decor_LookBacks not set, using default 0
Input Decor_Step not set, using default 0
Electron max energy 1.0 exceeded, increasing to 1.41 and trying again
Running with E Field 100.0 V/cm With Pressure 5250.434 tor

Input Decor_Colls not set, using default 0
Input Decor_LookBacks not set, using default 0
Input Decor_Step not set, using default 0

Electron max energy 1.0 exceeded, increasing to 1.41 and trying again

Running with E Field 100.0 V/cm With Pressure 6000.496 tor

Input Decor_Colls not set, us

Input Decor_LookBacks not set, using default 0
Input Decor_Step not set, using default 0


Running with E Field 220.0 V/cm With Pressure 3750.31 tor

Input Decor_Colls not set, using default 0
Input Decor_LookBacks not set, using default 0
Input Decor_Step not set, using default 0
Electron max energy 1.0 exceeded, increasing to 1.41 and trying again
Running with E Field 220.0 V/cm With Pressure 4500.372 tor

Input Decor_Colls not set, using default 0
Input Decor_LookBacks not set, using default 0
Input Decor_Step not set, using default 0

Running with E Field 220.0 V/cm With Pressure 5250.434 tor

Input Decor_Colls not set, using default 0
Input Decor_LookBacks not set, using default 0
Input Decor_Step not set, using default 0
Electron max energy 2.0 exceeded, increasing to 2.83 and trying again

Running with E Field 220.0 V/cm With Pressure 6000.496 tor

Input Decor_Colls not set, using default 0
Input Decor_LookBacks not set, using default 0
Input Decor_Step not set, using default 0


In [ ]:
Xenon=SingleGasMultiProcess("XENON",gpv1,"/home/argon/Downloads/Diffusion_Plot/",EFieldss,pp,NProcess,Delay=5,FileName="XENON_100_Mode2")

In [ ]:
manager=Manager()
#EFieldss=np.linspace(10,500,20)
#EFieldss=np.arange(100,1000,20)
EFieldss=np.arange(7000,1400,1000)
#pp=np.arange(5*760.062,8250.68,760.062)
InitialPressure=750.062 # 1 Bar
pp=[]
for i in range(5,11):
    pp.append(i*InitialPressure)
NProcess=20
print("-- ElectricFields --" )
print(EFieldss)
print("-- Pressures --" )
print(pp)


In [ ]:
#Xenon=SingleGasMultiProcess("XENON",gpv1,"/home/argon/Downloads/Diffusion_Plot/",EFieldss,pp,NProcess,Delay=5,FileName="XENON_100_HighField_iso")

In [9]:
manager=Manager()

gpv1   ={'Gases'              :['XENON'],
               'Fractions'             :[100],
               'Max_collisions'        :4e6,
               'EField_Vcm'            :500,
               'Max_electron_energy'   :0,
               'Temperature_C'         :23,
               'Pressure_Torr'         :7500.62,
               'BField_Tesla'          :0,
               'BField_angle'          :0,
               'Angular_dist_model'    :0, # 0 is Isotropic Scattering
               'Enable_penning'        :0,
               'Enable_thermal_motion' :1,
               'ConsoleOutputFlag'     :0,
                'NumSamples'            :20}

In [10]:
EFieldss=np.array([50])
pp=np.array([206.86,310.29,413.72,517.15,620.58,724.01,827.44])

In [1]:
#GasMixturesMultiProcess("ARGON","CH4",gpv1,"/home/ilker/CRAB/",EFieldss,pp,7,10,2)


In [2]:
#kk=PickleITRead("/home/ilker/CRAB/Diffusion_ARGON_90_CH4_10.p")